# Import package

In [2]:
#Import des packages généraux et utilitaires
import numpy as np
import pandas as pd

#Textmining
import re

#joblib
from joblib import dump,load

# Encodage html
import html
from bs4 import BeautifulSoup

# encodage des regex
#!pip install category_encoders
import category_encoders as ce
from category_encoders.helmert import HelmertEncoder

#!python -m pip install --upgrade pandas
#!pip install "pandas<2.0.0"

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/b7/f8/32d6b5aa4c4bc045fa2c4c58f88c325facc54721956c6313f0afea8ea853/pandas-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-2.1.0-cp311-cp311-win_amd64.whl.metadata (18 kB)
Using cached pandas-2.1.0-cp311-cp311-win_amd64.whl (11.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\Frederic\\conda3\\Lib\\site-packages\\~7ndas\\_libs\\algos.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.0
    Uninstalling pandas-2.1.0:
      Successfully uninstalled pandas-2.1.0


# Fonctions

## Ajouter les colonnes contenant le nombre d'occurrence des regex et les catégoriser

In [ ]:
#Fonction pour créer les colonnes contenant le nombre d'occurrence des regex à partir du  Dataframe
def add_regex_columns(df, dic_regex):
    df['mots_concat'] = df['mots'].apply(' '.join)
    for name_regex, regex in dic_regex.items():
        print("Création de la colonne pour le regex :", regex)
        def find_matching_units(text):
            return re.findall(regex, text, re.IGNORECASE)
        
        column_name_regex = f'{name_regex}'
        column_name_len = f'Nb_REGEX_{name_regex}'
        
        df[column_name_len] = df['mots_concat'].apply(find_matching_units).apply(lambda x: len(x))    
    return df

In [ ]:
# Fonction de catégorisation du nombre de REGEX
# 2 catégories (présence /absence)
def categorisation_2(x):
    if x>= 0 and x <= 1:
        x = 0
    if x>=2:
        x = 1
    return x

# variante
def categorisation_2_variante(x):
    if x>=1:
        x = 1
    return x

# 6 catégories (présence /absence)
def categorisation_6(x):
    if x >= 3 and x <= 4:
        x = 3
    if x >4 and x<=6:
        x = 4
    if x > 7:
        x = 5
    return x

In [ ]:
# A voir 
#colonnes à supprimer
liste_suppression = ['productid', 'imageid', 'prdtypecode', 'langue_texte', 'mots_stem', 'mots_stem_sans_chiffres',
       'mots_lem', 'mots_lem_sans_chiffres', 'mots_lem_stem','mots_lem_stem_sans_chiffres']


## Ajouter les colonnes encodées par REGEX, selon différentes variantes

In [4]:
# Fonctions pour ajouter les colonnes de REGEX encodés à l'issue du frequency encoding 
# sans catégorisation
def ajouter_colonnes_regex_frequency_encoding_brut(df):
    print("début du frequency encoding brut")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        encoder = (df.groupby(column_name_len).size()) / len(df)
        nom_encod = f"{name_regex}_encod_freq"
        df.loc[:, nom_encod] = df[column_name_len].map(encoder)
        if column_name_len in df.columns:
            df = df.drop(column_name_len,axis=1)
    df = df.drop(liste_suppression, axis=1)                  
    return df

#En créant au préalable deux catégories 0 et 1 (absence/présence)
def ajouter_colonnes_regex_frequency_encoding_2categories(df):
    print("début du frequency encoding 2 catégories")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        unite_cat = f"{name_regex}_cat"
        df[unite_cat] = df[column_name_len].apply(lambda x : categorisation_2(x)) # appliquer la catégorisation à deux niveaux
        encoder = (df.groupby(unite_cat).size()) / len(df)
        nom_encod = f"{name_regex}_encod_freq_2cat"
        df.loc[:, nom_encod] = df[unite_cat].map(encoder)   
        if column_name_len in df.columns:
            df = df.drop(column_name_len,axis=1)
        if unite_cat in df.columns:
            df = df.drop(unite_cat,axis=1)  
    df = df.drop(liste_suppression , axis=1) 
    return df      

#En créant au préalable deux catégories 0 et 1 (absence/présence) (variante)
def ajouter_colonnes_regex_frequency_encoding_2categories_variante(df):
    print("début du frequency encoding 2 catégories")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        unite_cat = f"{name_regex}_cat"
        df[unite_cat] = df[column_name_len].apply(lambda x : categorisation_2_variante(x)) # appliquer la catégorisation à deux niveaux
        encoder = (df.groupby(unite_cat).size()) / len(df)
        nom_encod = f"{name_regex}_encod_freq_2cat_variante"
        df.loc[:, nom_encod] = df[unite_cat].map(encoder)   
        if column_name_len in df.columns:
            df = df.drop(column_name_len,axis=1)
        if unite_cat in df.columns:
            df = df.drop(unite_cat,axis=1)  
    df = df.drop(liste_suppression , axis=1) 
    return df      
         
    
# En créant au préalable des catégories selon les quartiles
def ajouter_colonnes_regex_frequency_encoding_quartiles_categories(df):
    print("début du frequency encoding catégories par quartiles")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        unite_cat = f"{name_regex}_cat"
        def categorisation_REGEX_quartiles(x,df):
            premier_quartile = df[column_name_len].quantile(q=0.25)
            second_quartile = df[column_name_len].quantile(q=0.5)
            troisieme_quartile = df[column_name_len].quantile(q=0.75)
            if x>= 0 and x < premier_quartile:
                x = 0
            if x >= premier_quartile and x < second_quartile:
                x=1
            if x >= second_quartile and x < troisieme_quartile:
                x=2
            if x >= troisieme_quartile:
                x=3
            return x         
        df[unite_cat] = df[column_name_len].apply(lambda x : categorisation_REGEX_quartiles(x,df)) # appliquer la catégorisation par quartiles à deux niveaux
        encoder = (df.groupby(unite_cat).size()) / len(df)
        nom_encod = f"{name_regex}_encod_freq_quartiles"
        df.loc[:, nom_encod] = df[unite_cat].map(encoder)
        if column_name_len in df.columns:
            df = df.drop(column_name_len,axis=1)
        if unite_cat in df.columns:
            df = df.drop(unite_cat,axis=1)        
    df = df.drop(liste_suppression, axis=1) 
    return df

         
# Fonction pour ajouter les colonnes de REGEX encodés à l'issue du Helmert encoding
# Ce type de codage peut être utile dans certaines situations où les niveaux de la variable catégorielle sont ordonnés,
# par exemple, du plus bas au plus haut ou du plus petit au plus grand. Il compare la moyenne d'un niveau à celle des autres niveaux

#Avec deux catégories
def ajouter_colonnes_regex_helmert_encoding_2categories(df):
    print("début du Helmert encoding catégories par quartiles")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        unite_cat = f"{name_regex}_cat"
        df[unite_cat] = df[column_name_len].apply(lambda x : categorisation_2(x)) # appliquer la catégorisation à 6 niveaux
        obj_df = df.copy() # Préparer un tout nouveau df
        encoder = ce.HelmertEncoder(cols=[unite_cat]) #objet Helmert encoder
        df_he = encoder.fit_transform(obj_df[unite_cat])
        df = pd.concat([df,df_he],axis='columns') # concatener les dfs
        if column_name_len in df.columns:
            df = df.drop(column_name_len,axis=1)
        if unite_cat in df.columns:
            df = df.drop(unite_cat,axis=1)
        if 'intercept'in df.columns:
            df = df.drop('intercept',axis=1) 
    df = df.drop(liste_suppression, axis=1) 
    return df         
   
         
#Avec 6 catégories
def ajouter_colonnes_regex_helmert_encoding_6cat(df):
    print("début du Helmert encoding catégories 6 catégories")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        unite_cat = f"{name_regex}_cat"
        df[unite_cat] = df[column_name_len].apply(lambda x : categorisation_6(x)) # appliquer la catégorisation à 6 niveaux
        obj_df = df.copy() # Préparer un tout nouveau df
        encoder = ce.HelmertEncoder(cols=[unite_cat]) #objet Helmert encoder
        df_he = encoder.fit_transform(obj_df[unite_cat])
        df = pd.concat([df,df_he],axis='columns') # concatener les dfs 
        if column_name_len in df.columns:
            df = df.drop(column_name_len,axis=1)
        if unite_cat in df.columns:
            df = df.drop(unite_cat,axis=1)
        if 'intercept'in df.columns:
            df = df.drop('intercept',axis=1)          
    df = df.drop(liste_suppression, axis=1)
    return df  

# Fonction pour ajouter les colonnes de REGEX encodés à l'issue du BackwardDifference encoding
# En créant au préalable 2 catégories 
def ajouter_colonnes_BackwardDifference_encoding_2cat(df):
    print("début du BackwardDifference encoding catégories par quartiles")
    for name_regex, regex in dic_regex.items():
        column_name_len = f'Nb_REGEX_{name_regex}'
        unite_cat = f"{name_regex}_cat"
        df[unite_cat] = df[column_name_len].apply(lambda x : categorisation_2(x)) # appliquer la catégorisation à 6 niveaux
        obj_df = df.copy() # Préparer un tout nouveau df
        encoder = ce.BackwardDifferenceEncoder(cols=[unite_cat]) #object Helmert encoder
        df_bd = encoder.fit_transform(obj_df[unite_cat])
        df = pd.concat([df,df_bd],axis=1) # concatener les dfs
        return df


## Encodage des autres métadonnées

In [ ]:
def ajouter_colonne_encodage_description_vide(df):
    encoder = (df.groupby('description_vide').size()) / len(df)
    nom_encod = "description_vide_encod_freq"
    df.loc[:, nom_encod] = df['description_vide'].map(encoder)
    df =  df.drop('description_vide',axis=1)
    return df

def ajouter_colonne_encodage_nb_mots(df):
    nb_mots = "Nb_mots"
    df.loc[:, nb_mots] = df['mots'].apply(lambda x : len(x))
    def categorisation_quartiles(x,df):
        premier_quartile = df[nb_mots].quantile(q=0.25)
        second_quartile = df[nb_mots].quantile(q=0.5)
        troisieme_quartile = df[nb_mots].quantile(q=0.75)
        if x>= 0 and x< premier_quartile:
            x = 0
        if x >= premier_quartile and x < second_quartile:
            x=1
        if x >= second_quartile and x <  troisieme_quartile:
            x=2
        if x >= troisieme_quartile:
            x=3
        return x
    df['Nb_mots_cat'] = df[nb_mots].apply(lambda x : categorisation_quartiles(x,df))
    encoder = (df.groupby('Nb_mots_cat').size()) / len(df)
    nom_encod = "Nb_mots_cat_encod_freq"
    df.loc[:, nom_encod] = df['Nb_mots_cat'].map(encoder)   
    df=df.drop(['Nb_mots','Nb_mots_cat'],axis=1)
    return df

# Liste des Regex

In [5]:
densite = r"(?i)\d*\s*\b(kg\/m³|kg\/m3|g\/m³|g\/m3|\d+kg\/m³|\d+kg\/m3|\d+g\/m³|\d+g\/m3)s?\b"

masse =r"(?i)\d*\s*\b(kg|g|gr|t|mg|tonnes?|kilos?|microgrammes?|grammes?|milligrammes?|µg|lbs?|pounds?|\d+g|\d+kg|\d+gr|\d+mg|\d+t|\d+tonnes?|\d+kilos?|\d+microgrammes?|\d+grammes?|\d+milligrammes?|\d+µg|\d+lbs?|\d+pounds?)\b"

surface = r"(?i)\d*s*\b(m²|m2|metres?2|cm²|centimetres?2|cm2|km²|kilometres?2|km2|ft²|ft2|in²|in2|ha|ac|\d+m²|\d+m2|\d+metres?2|\d+cm²|\d+centimetres?2|\d+cm2|\d+km²|\d+kilometres?2|\d+km2|\d+ft²|\d+ft2|\d+in²|d+in2|\d+ha|\d+ac)\b"

vitesse = r"(?i)\d*\s*\b(m\/s|km\/h|mph|m.s-1|\d+m\/s|\d+km\/h|\d+mph|\d+m.s-1)\b"

pression = r"(?i)\d*\s*\b(Pa|Pascal|bar|atm|mmHg|\d+Pa|\d+bar|\d+Pascal|\d+atm|\d+mmHg)\b"

energie_alim = r"\d*\s*\b(J|kJ|cal|kcal|CAL|KCAL|joules?|Joules?|kilojoules?|Calories?|calories?|kilocalories?|\d+J|\d+kJ|\d+cal|\d+kcal|\d+kCAL|\d+KCAL|\d+Joules?|\d+Calories?|\d+joules?|d+kilojoules?|d+calories?|d+kilocalories?)\b"

energie_elec = r"(?i)\d*\s*\b(kW|W|Watts?|kWh|\d+kWh|kiloWatts?|\d+kW|\d+W|\d+Watts?|\d+kiloWatts?)\b"

longueur = r"(?i)\d*\s*\b(centimètres?|centimetres?|cm|foot|ft|inch|kilomètres?|kilometres?|km|m|micromètres?|micrometres?|μm|millimètres?|millimetres?|mm|nanomètres?|nanometres?|nm|pouce|yard|yd|d+centimètres?|d+centimetres?|d+cm|d+foot|\d+ft|\d+inch|\d+kilomètres?|\d+kilometres?|\d+km|\d+m|micromètres?|\d+micrometres?|\d+μm|millimètres?|millimetres?|\d+mm|\d+nanomètres?|\d+nanometres?|\d+nm|\d+pouce|\d+yard|\d+yd)s?\b"

volume = r"(?i)\d*\s*\b(litre|l|millilitres?|ml|centilitres?|cl|mètre cube|metre cube|m3|m 3|centimètres?cube|centimetres? cube|cm3|cm 3|pied cube|ft 3|ft3|pouce cube|in3|in 3|gallon|gal|once liquide|fl oz|\d+litre|\d+l|\d+millilitres?|\d+ml|\d+centilitres?|\d+cl|\d+mètre cube|\d+metre cube|\d+m3|\d+m 3|\d+centimètres? cube|\d+centimetres? cube|\d+cm3|\d+cm 3|\d+pied cube|\d+ft 3|\d+ft3|\d+pouce cube|\d+in3|\d+in 3|\d+gallon|\d+gal|\d+once liquide|\d+fl oz)s?\b"

memoire = r"(?i)\d*\s*\b(hertz|hz|kilohertz|khz|megahertz|mhz|gigahertz|ghz|terahertz|thz|petahertz|phz|exahertz|ehz|zettahertz|zhz|yottahertz|yhz|bits?|bytes?|kilobits?|kb|kilobytes?|kb|ko|mmegabits?|mb|megabytes?|mb|gigabits?|gb|gigabytes?|gb|terabits?|tb|terabytes?|tb|petabits?|pb|petabytes?|pb|exabits?|eb|exabytes?|eb|zettabits?|zb|zettabytes?|zb|yottabits?|yb|yottabytes?|yb|octets?|mégaoctets?|megaoctets?|mega octets?|mo|giga-octets?|giga octets?|go|téraoctets?|teraoctets?|tera octets?|to|\d+hertz|\d+hz|\d+kilohertz|\d+khz|\d+megahertz|\d+mhz|\d+gigahertz|\d+ghz|\d+terahertz|\d+thz|\d+petahertz|\d+phz|\d+exahertz|\d+ehz|\d+zettahertz|\d+zhz| \d+yottahertz|\d+yhz|\d+bits?|\d+bytes?|\d+kilobits?|\d+kb|\d+kilobytes?|\d+kb|\d+megabits?|\d+mb| \d+octets?|\d+megabytes?|\d+mb|\d+gigabits?|\d+gb|\d+ko|\d+gigabytes?|\d+gb|\d+terabits?|\d+tb|\d+terabytes?|\d+tb| \d+petabits?|\d+pb|petabytes?|\d+pb|\d+exabits?|\d+eb|\d+exabytes?|\d+eb|\d+zettabits?|\d+zb|\d+zettabytes?|\d+zb|\d+yottabits?|\d+yb|\d+yottabytes?|\d+yb|\d+mégaoctets?|\d+megaoctets?|\d+mega octets?|\d+mo|\d+giga-octets?|\d+giga octets?|\d+go|\d+téraoctets?|\d+teraoctets?|\d+tera octets?|\d+to)s?\b"

temps = r"(?i)\d*\s*\b(secondes?|s|millisecondes?|ms|microsecondes?|μs|nanosecondes?|ns|picosecondes?|ps|minutes?|min|heures?|h|jours?|j|semaines?|sem|mois|années?|ans?|décennies?|siècles?|siè|millénaires?|\d+secondes?|\d+s|\d+millisecondes?|\d+ms|\d+microsecondes?|\d+μs|\d+nanosecondes?|\d+ns|\d+picosecondes?|\d+ps|\d+minutes?|\d+min|heures?|\d+h|\d+jours?|\d+j|\d+semaines?|\d+sem|\d+mois|\d+années?|\d+ans?|\d+décennies?|\d+siècles?|\d+siè|\d+millénaires?)s?\b"

chiffres = r"[0-9]+"

dic_regex = {'densite' : densite ,'masse': masse,'surface' : surface,'vitesse' : vitesse,'pression' : pression,'energie_alim' : energie_alim,'energie_elec': energie_elec,'longueur' : longueur,'volume' : volume,'memoire': memoire,'temps' : temps, 'chiffres':chiffres}

# Fonctions pour créer les colonnes d'encodage à partir du df initial

In [6]:
# Création des différents df avec variations d'encodage à l'aide de fonctions

# Frequency Encodage simple
def creation_frequency_encodage_REGEX(df):
    df=add_regex_columns(df,dic_regex)
    df = ajouter_colonnes_regex_frequency_encoding_brut(df)
    df = ajouter_colonne_encodage_description_vide(df)
    df = ajouter_colonne_encodage_nb_mots(df)
    return df

# Frequency Encodage 2 catégories
def creation_frequency_encodage_REGEX_2cat(df):
    df=add_regex_columns(df,dic_regex)
    df = ajouter_colonnes_regex_frequency_encoding_2categories(df)
    df = ajouter_colonne_encodage_description_vide(df)
    df = ajouter_colonne_encodage_nb_mots(df)
    return df

# Frequency Encodage 2 catégories (variante)
def creation_frequency_encodage_REGEX_2cat_variante(df):
    df=add_regex_columns(df,dic_regex)
    df = ajouter_colonnes_regex_frequency_encoding_2categories_variante(df)
    df = ajouter_colonne_encodage_description_vide(df)
    df = ajouter_colonne_encodage_nb_mots(df)
    return df

# Frequency Encodage selon les quartiles
def creation_frequency_encodage_REGEX_quartiles(df):
    df=add_regex_columns(df,dic_regex)
    df = ajouter_colonnes_regex_frequency_encoding_quartiles_categories(df)
    df = ajouter_colonne_encodage_description_vide(df)
    df = ajouter_colonne_encodage_nb_mots(df)
    return df

# Helmert Encodage 2 catégories
def creation_Helmert_encodage_REGEX_cat2(df):
    df= add_regex_columns(df,dic_regex)
    df = ajouter_colonnes_regex_helmert_encoding_2categories(df)
    df = ajouter_colonne_encodage_description_vide(df)
    df = ajouter_colonne_encodage_nb_mots(df)
    dict = {'masse_cat_0':'masse_encod_Helmert_2cat_0', 'surface_cat_0':'surface_encod_Helmert_2cat_0',
       'vitesse_cat_0':'vitesse_encod_Helmert_2cat_0', 'pression_cat_0':'pression_encod_Helmert_2cat_0', 'energie_alim_cat_0':'energie_alim_encod_Helmert_2cat_0',
       'energie_elec_cat_0':'masse_encod_Helmert_2cat_0', 'longueur_cat_0':'longueur_encod_Helmert_2cat_0', 'volume_cat_0':'volume_encod_Helmert_2cat_0', 'memoire_cat_0':'memoire_encod_Helmert_2cat_0',
       'temps_cat_0':'temps_encod_Helmert_2cat_0', 'chiffres_cat_0':'chiffres_encod_Helmert_2cat_0'}
    df=df.rename(columns=dict)
    return df


# Helmert Encodage 6 catégories
def creation_Helmert_encodage_REGEX_6cat(df):
    df= add_regex_columns(df,dic_regex)
    df = ajouter_colonnes_regex_helmert_encoding_6cat(df)
    df = ajouter_colonne_encodage_description_vide(df)
    df = ajouter_colonne_encodage_nb_mots(df)
    dict2 = {'masse_cat_0':'masse_encod_Helmert_6cat_0', 'surface_cat_0':'surface_encod_Helmert_6cat_0', 'vitesse_cat_0':'vitesse_encod_Helmert_6cat_0', 'pression_cat_0':'pression_encod_Helmert_6cat_0', 'energie_alim_cat_0':'energie_alim_encod_Helmert_6cat_0',
       'energie_elec_cat_0':'energie_encod_Helmert_6cat_0', 'longueur_cat_0':'longueur_encod_Helmert_6cat_0', 'volume_cat_0':'volume_encod_Helmert_6cat_0', 'memoire_cat_0':'memoire_encod_Helmert_6cat_0',
       'temps_cat_0':'temps_encod_Helmert_6cat_0', 'chiffres_cat_0':'chiffres_encod_Helmert_6cat_0','masse_cat_1':'masse_encod_Helmert_6cat_1', 'surface_cat_1':'surface_masse_encod_Helmert_6cat_1',
       'vitesse_cat_1':'vitesse_encod_Helmert_6cat_1', 'pression_cat_1':'pression_encod_Helmert_6cat_1', 'energie_alim_cat_1':'energie_alim_encod_Helmert_6cat_1',
       'energie_elec_cat_1':'energie_encod_Helmert_6cat_1', 'longueur_cat_1':'longueur_encod_Helmert_6cat_1', 'volume_cat_1':'volume_encod_Helmert_6cat_1', 'memoire_cat_1':'memoire_encod_Helmert_6cat_1',
       'temps_cat_1':'temps_encod_Helmert_6cat_1', 'chiffres_cat_1':'chiffres_encod_Helmert_6cat_1','masse_cat_2':'masse_encod_Helmert_6cat_2', 'surface_cat_2':'surface_encod_Helmert_6cat_2',
       'vitesse_cat_2':'vitesse_encod_Helmert_6cat_2', 'pression_cat_2':'pression_encod_Helmert_6cat_2', 'energie_alim_cat_2':'energie_alim_encod_Helmert_6cat_2',
       'energie_elec_cat_2':'energie_encod_Helmert_6cat_2', 'longueur_cat_2':'longueur_encod_Helmert_6cat_2', 'volume_cat_2':'volume_encod_Helmert_6cat_2', 'memoire_cat_2':'memoire_encod_Helmert_6cat_2',
       'temps_cat_2':'temps_encod_Helmert_6cat_2', 'chiffres_cat_2':'chiffres_encod_Helmert_6cat_2','masse_cat_3':'masse_encod_Helmert_6cat_3', 'surface_cat_13':'surface_masse_encod_Helmert_6cat_3',
       'vitesse_cat_3':'vitesse_encod_Helmert_6cat_3', 'pression_cat_3':'pression_encod_Helmert_6cat_3', 'energie_alim_cat_3':'energie_alim_encod_Helmert_6cat_3',
       'energie_elec_cat_3':'energie_encod_Helmert_6cat_3', 'longueur_cat_3':'longueur_encod_Helmert_6cat_3', 'volume_cat_3':'volume_encod_Helmert_6cat_3', 'memoire_cat_3':'memoire_encod_Helmert_6cat_3',
       'temps_cat_3':'temps_encod_Helmert_6cat_3', 'chiffres_cat_3':'chiffres_encod_Helmert_6cat_3','masse_cat_4':'masse_encod_Helmert_6cat_4', 'surface_cat_4':'surface_masse_encod_Helmert_6cat_4','vitesse_cat_4':'vitesse_encod_Helmert_6cat_4', 'pression_cat_4':'pression_encod_Helmert_6cat_4', 'energie_alim_cat_4':'energie_alim_encod_Helmert_6cat_4',
       'energie_elec_cat_4':'energie_encod_Helmert_6cat_4', 'longueur_cat_4':'longueur_encod_Helmert_6cat_4', 'volume_cat_4':'volume_encod_Helmert_6cat_4', 'memoire_cat_4':'memoire_encod_Helmert_6cat_4',
       'temps_cat_4':'temps_encod_Helmert_6cat_4', 'chiffres_cat_4':'chiffres_encod_Helmert_6cat_4','masse_cat_5':'masse_encod_Helmert_6cat_5', 'surface_cat_5':'surface_masse_encod_Helmert_6cat_5','vitesse_cat_5':'vitesse_encod_Helmert_6cat_5', 'pression_cat_5':'pression_encod_Helmert_6cat_5', 'energie_alim_cat_5':'energie_alim_encod_Helmert_6cat_5',
       'energie_elec_cat_5':'energie_encod_Helmert_6cat_5', 'longueur_cat_5':'longueur_encod_Helmert_6cat_5', 'volume_cat_5':'volume_encod_Helmert_6cat_5', 'memoire_cat_5':'memoire_encod_Helmert_6cat_5',
       'temps_cat_5':'temps_encod_Helmert_6cat_5', 'chiffres_cat_5':'chiffres_encod_Helmert_6cat_5'}        
    df=df.rename(columns=dict2)
    return df       
    

In [7]:

df = load('df_train_post_trad.joblib')

## Creation des colonnes d'encodage à partir des fonctions

In [21]:
#Chargement du df initial
df = load('df_train_post_trad.joblib')

df_freq_encoding= creation_frequency_encodage_REGEX(df)
df_freq_encoding_2cat= creation_frequency_encodage_REGEX_2cat(df)
df_freq_encoding_2cat_variante = creation_frequency_encodage_REGEX_2cat_variante(df)
df_freq_encoding_quartiles = creation_frequency_encodage_REGEX_quartiles(df)
df_Helmert_encoding_cat2 = creation_Helmert_encodage_REGEX_cat2(df)
df_Helmert_encoding_cat6 = creation_Helmert_encodage_REGEX_6cat(df)

(?i)\d*\s*\b(kg\/m³|kg\/m3|g\/m³|g\/m3|\d+kg\/m³|\d+kg\/m3|\d+g\/m³|\d+g\/m3)s?\b
(?i)\d*\s*\b(kg|g|gr|t|mg|tonnes?|kilos?|microgrammes?|grammes?|milligrammes?|µg|lbs?|pounds?|\d+g|\d+kg|\d+gr|\d+mg|\d+t|\d+tonnes?|\d+kilos?|\d+microgrammes?|\d+grammes?|\d+milligrammes?|\d+µg|\d+lbs?|\d+pounds?)\b
(?i)\d*s*\b(m²|m2|metres?2|cm²|centimetres?2|cm2|km²|kilometres?2|km2|ft²|ft2|in²|in2|ha|ac|\d+m²|\d+m2|\d+metres?2|\d+cm²|\d+centimetres?2|\d+cm2|\d+km²|\d+kilometres?2|\d+km2|\d+ft²|\d+ft2|\d+in²|d+in2|\d+ha|\d+ac)\b
(?i)\d*\s*\b(m\/s|km\/h|mph|m.s-1|\d+m\/s|\d+km\/h|\d+mph|\d+m.s-1)\b
(?i)\d*\s*\b(Pa|Pascal|bar|atm|mmHg|\d+Pa|\d+bar|\d+Pascal|\d+atm|\d+mmHg)\b
\d*\s*\b(J|kJ|cal|kcal|CAL|KCAL|joules?|Joules?|kilojoules?|Calories?|calories?|kilocalories?|\d+J|\d+kJ|\d+cal|\d+kcal|\d+kCAL|\d+KCAL|\d+Joules?|\d+Calories?|\d+joules?|d+kilojoules?|d+calories?|d+kilocalories?)\b
(?i)\d*\s*\b(kW|W|Watts?|kWh|\d+kWh|kiloWatts?|\d+kW|\d+W|\d+Watts?|\d+kiloWatts?)\b
(?i)\d*\s*\b(centimètres?|centimet

(?i)\d*\s*\b(litre|l|millilitres?|ml|centilitres?|cl|mètre cube|metre cube|m3|m 3|centimètres?cube|centimetres? cube|cm3|cm 3|pied cube|ft 3|ft3|pouce cube|in3|in 3|gallon|gal|once liquide|fl oz|\d+litre|\d+l|\d+millilitres?|\d+ml|\d+centilitres?|\d+cl|\d+mètre cube|\d+metre cube|\d+m3|\d+m 3|\d+centimètres? cube|\d+centimetres? cube|\d+cm3|\d+cm 3|\d+pied cube|\d+ft 3|\d+ft3|\d+pouce cube|\d+in3|\d+in 3|\d+gallon|\d+gal|\d+once liquide|\d+fl oz)s?\b
(?i)\d*\s*\b(hertz|hz|kilohertz|khz|megahertz|mhz|gigahertz|ghz|terahertz|thz|petahertz|phz|exahertz|ehz|zettahertz|zhz|yottahertz|yhz|bits?|bytes?|kilobits?|kb|kilobytes?|kb|ko|mmegabits?|mb|megabytes?|mb|gigabits?|gb|gigabytes?|gb|terabits?|tb|terabytes?|tb|petabits?|pb|petabytes?|pb|exabits?|eb|exabytes?|eb|zettabits?|zb|zettabytes?|zb|yottabits?|yb|yottabytes?|yb|octets?|mégaoctets?|megaoctets?|mega octets?|mo|giga-octets?|giga octets?|go|téraoctets?|teraoctets?|tera octets?|to|\d+hertz|\d+hz|\d+kilohertz|\d+khz|\d+megahertz|\d+mhz|\d+

C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be add

(?i)\d*\s*\b(kg\/m³|kg\/m3|g\/m³|g\/m3|\d+kg\/m³|\d+kg\/m3|\d+g\/m³|\d+g\/m3)s?\b
(?i)\d*\s*\b(kg|g|gr|t|mg|tonnes?|kilos?|microgrammes?|grammes?|milligrammes?|µg|lbs?|pounds?|\d+g|\d+kg|\d+gr|\d+mg|\d+t|\d+tonnes?|\d+kilos?|\d+microgrammes?|\d+grammes?|\d+milligrammes?|\d+µg|\d+lbs?|\d+pounds?)\b
(?i)\d*s*\b(m²|m2|metres?2|cm²|centimetres?2|cm2|km²|kilometres?2|km2|ft²|ft2|in²|in2|ha|ac|\d+m²|\d+m2|\d+metres?2|\d+cm²|\d+centimetres?2|\d+cm2|\d+km²|\d+kilometres?2|\d+km2|\d+ft²|\d+ft2|\d+in²|d+in2|\d+ha|\d+ac)\b
(?i)\d*\s*\b(m\/s|km\/h|mph|m.s-1|\d+m\/s|\d+km\/h|\d+mph|\d+m.s-1)\b
(?i)\d*\s*\b(Pa|Pascal|bar|atm|mmHg|\d+Pa|\d+bar|\d+Pascal|\d+atm|\d+mmHg)\b
\d*\s*\b(J|kJ|cal|kcal|CAL|KCAL|joules?|Joules?|kilojoules?|Calories?|calories?|kilocalories?|\d+J|\d+kJ|\d+cal|\d+kcal|\d+kCAL|\d+KCAL|\d+Joules?|\d+Calories?|\d+joules?|d+kilojoules?|d+calories?|d+kilocalories?)\b
(?i)\d*\s*\b(kW|W|Watts?|kWh|\d+kWh|kiloWatts?|\d+kW|\d+W|\d+Watts?|\d+kiloWatts?)\b
(?i)\d*\s*\b(centimètres?|centimet

C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
C:\Users\Frederic\conda3\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be add

### Enregistrement

In [22]:
dump(df_freq_encoding,'df_freq_encoding.joblib')
dump(df_freq_encoding_2cat,'df_freq_encoding_2cat.joblib')
dump(df_freq_encoding_quartiles,'df_freq_encoding_quartiles.joblib')
dump(df_Helmert_encoding_cat2,'df_helmert_encoding_cat2.joblib')
dump(df_Helmert_encoding_cat6,'df_helmert_encoding_cat6.joblib')

['df_helmert_encoding_cat6.joblib']

In [40]:
dump(df_freq_encoding_2cat_variante,'df_freq_encoding_2cat_variante.joblib')

['df_freq_encoding_2cat_variante.joblib']

### Chargement

In [3]:
df_freq_encoding = load('df_freq_encoding.joblib')
df_freq_encoding_2cat = load('df_freq_encoding_2cat.joblib')
df_freq_encoding_2cat_variante = load('df_freq_encoding_2cat_variante.joblib')
df_freq_encoding_quartiles = load('df_freq_encoding_quartiles.joblib')
df_helmert_encoding_cat2 = load('df_helmert_encoding_cat2.joblib')
df_Helmert_encoding_cat6 = load('df_Helmert_encoding_cat6.joblib')



FileNotFoundError: [Errno 2] No such file or directory: 'df_freq_encoding.joblib'

## Traitement divers

In [43]:
# Suppression des colonnes inutiles
for df in [df_freq_encoding,df_freq_encoding_2cat,df_freq_encoding_quartiles,df_helmert_encoding_cat2,df_Helmert_encoding_cat6,df_freq_encoding_2cat_variante]:
    df=df.drop(['mots','mots_concat'],axis=1)

for df in [df_freq_encoding_2cat,df_freq_encoding_quartiles,df_helmert_encoding_cat2,df_Helmert_encoding_cat6,df_freq_encoding_2cat_variante]:
    df=df.drop(['description_vide_encod_freq','Nb_mots_cat_encod_freq'],axis=1)

df = load('df_train_post_trad.joblib')
target = df['prdtypecode']
df_freq_encoding = pd.concat([df_freq_encoding,target],axis='columns')

In [45]:
# Extraction des listes de colonnes 
Col_df_freq_encoding = list(df_freq_encoding.columns)
Col_df_freq_encoding_2cat = list(df_freq_encoding_2cat.columns)
Col_df_freq_encoding_2cat_variante = list(df_freq_encoding_2cat_variante)
Col_df_freq_encoding_quartiles= list(df_freq_encoding_quartiles.columns)
Col_df_helmert_encoding_cat2 = list(df_helmert_encoding_cat2.columns)
Col_df_Helmert_encoding_cat6= list(df_Helmert_encoding_cat6.columns)
Col_df_helmert_encoding_cat2 = list(df_Helmert_encoding_cat2.columns)
Col_df_helmert_encoding_cat6= list(df_Helmert_encoding_cat6.columns)


In [47]:
# Concaténation de l'ensebmle des varaintes de df crées
df_REGEX_encode = pd.concat([df_freq_encoding,df_freq_encoding_2cat,df_freq_encoding_quartiles,
                             df_helmert_encoding_cat2,df_Helmert_encoding_cat6,df_freq_encoding_2cat_variante],axis='columns')

#Suppression des colonnes 'mots','mots_concat'
df_REGEX_encode = df_REGEX_encode.drop(['mots','mots_concat'],axis=1)

#Suppression des colonnes en doublons
df_REGEX_encode  = df_REGEX_encode .loc[:,~df_REGEX_encode .columns.duplicated()]


In [50]:
# Sauegarde du df concaténé pour la prochaine étape
dump(df_REGEX_encode,'df_REGEX_encode.joblib')

['df_REGEX_encode.joblib']